In [18]:
from datasets import load_dataset
data = load_dataset('arbml/EASC')

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset parquet (/home/codespace/.cache/huggingface/datasets/arbml___parquet/arbml--EASC-85636dd59f0cb935/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 1/1 [00:00<00:00,  4.66it/s]


In [19]:
data

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 153
    })
})

In [20]:
split_data = data['train'].train_test_split(test_size = 0.1, seed = 42)

Loading cached split indices for dataset at /home/codespace/.cache/huggingface/datasets/arbml___parquet/arbml--EASC-85636dd59f0cb935/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-8fd5401a7de88420.arrow and /home/codespace/.cache/huggingface/datasets/arbml___parquet/arbml--EASC-85636dd59f0cb935/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d5eb51f48ff3fa2a.arrow


In [21]:
split_data

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 137
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 16
    })
})

In [48]:
split_data

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 137
    })
    test: Dataset({
        features: ['article', 'summary'],
        num_rows: 16
    })
})

In [51]:
sys_msg = "Summarize the following statement to Arabic"
def create_chat_prompt(sys_msg, input_text):
    return [
        {"role": "system", "content": sys_msg}, 
        {"role": "user", "content": input_text}
    ]

def create_chat_example(article, summary):
    return [
        {"role": "system", "content": article, "name": "example_user"},
        {"role": "system", "content": summary, "name": "example_assistant"},
    ]

In [23]:
!mkdir ../registry/data/summarization

test.jsonl  train.jsonl


In [57]:
dev_df = split_data['train'].to_pandas()
dev_df["sample"] = dev_df.apply(lambda x: create_chat_example(x['article'], x['summary']), axis=1)
dev_df[["sample"]].to_json('../registry/data/summarization/few_shot.jsonl', lines=True, orient="records")

test_df = split_data['test'].to_pandas()
test_df["input"] = test_df['article'].apply(lambda x: create_chat_prompt(sys_msg, x))
test_df["ideal"] = test_df['summary']
test_df[["input", "ideal"]].to_json('../registry/data/summarization/samples.jsonl', lines=True, orient="records")

In [70]:
dev_df

,article,summary,sample
0,﻿قضية لوكربي هي قضية جنائية ترتبت على سقوط طائ...,اصرت الدولتان على طلبهما ورفضت ليبيا الاستجاب...,"[{'role': 'system', 'content': '﻿قضية لوكربي ه..."
1,﻿الحرب الباردة اصطلاح يطلق على العلاقات الجديد...,الحرب الباردة اصطلاح يطلق على العلاقات الجديدة...,"[{'role': 'system', 'content': '﻿الحرب الباردة..."
2,﻿الألعاب الأولمبية أو الأولمبياد حدث رياضي عال...,الالعاب الاولمبية او الاولمبياد حدث رياضي عالم...,"[{'role': 'system', 'content': '﻿الألعاب الأول..."
3,﻿حرب فيتنام أو الحرب الهندوصينية الثانية كانت ...,وقد عُرف هؤلاء بالفيت كونغ.\nورويداً رويداً تش...,"[{'role': 'system', 'content': '﻿حرب فيتنام أو..."
4,﻿سفينة وسيلة نقل عامة فوق الماء إستخدمها الإنس...,سفينة وسيلة نقل عامة فوق الماء استخدمها الانسا...,"[{'role': 'system', 'content': '﻿سفينة وسيلة ن..."
...,...,...,...
132,﻿الدين البهائي هو دين عالمي مستقل ينسبه البعض ...,الدين البهائي هو دين عالمي مستقل ينسبه البعض ل...,"[{'role': 'system', 'content': '﻿الدين البهائي..."
133,﻿حفلت المقابلة الصحفية المستفيضة التي اجرتها ص...,حفلت المقابلة الصحفية المستفيضة التي اجرتها صح...,"[{'role': 'system', 'content': '﻿حفلت المقابلة..."
134,﻿تعتبر بطولة كأس العالم لكرة القدم أهم مسابقة ...,تعتبر بطولة كاس العالم لكرة القدم اهم مسابقة ي...,"[{'role': 'system', 'content': '﻿تعتبر بطولة ك..."
135,﻿زحل هو الكوكب السادس في النظام الشمسي ويتميز ...,زحل هو الكوكب السادس في النظام الشمسي ويتميز ب...,"[{'role': 'system', 'content': '﻿زحل هو الكوكب..."


In [3]:
import os
os.environ["OPENAI_API_KEY"] = "<token>"

In [76]:
!oaieval gpt-3.5-turbo summarization --max_samples 1

[2023-04-02 20:37:18,900] [registry.py:156] Loading registry from /workspaces/evals/evals/registry/evals
[2023-04-02 20:37:18,985] [registry.py:156] Loading registry from /home/codespace/.evals/evals
[2023-04-02 20:37:19,455] [oaieval.py:213] Run started: 230402203719XRXEKU3U
[2023-04-02 20:37:20,116] [data.py:75] Fetching summarization/samples.jsonl
[2023-04-02 20:37:20,118] [eval.py:32] Evaluating 1 samples
[2023-04-02 20:37:20,121] [eval.py:152] Running in threaded mode with 10 threads!
100%|█████████████████████████████████████████████| 1/1 [00:11<00:00, 11.75s/it]
[2023-04-02 20:37:31,877] [record.py:320] Final report: {'average_rouge1': 0.2564102564102564, 'average_rougeL': 0.23076923076923078}. Logged to /tmp/evallogs/230402203719XRXEKU3U_gpt-3.5-turbo_summarization.jsonl
[2023-04-02 20:37:31,878] [oaieval.py:231] Final report:
[2023-04-02 20:37:31,878] [oaieval.py:233] average_rouge1: 0.2564102564102564
[2023-04-02 20:37:31,878] [oaieval.py:233] average_rougeL: 0.23076923076923

In [68]:
!ls /tmp/evallogs

2304021911382IFW6V2A_gpt-3.5-turbo_summarization.jsonl
2304021912367U2S3TIC_gpt-3.5-turbo_summarization.jsonl
230402191315OPBQRLMU_gpt-3.5-turbo_summarization.jsonl
230402191540AXPX7PME_gpt-3.5-turbo_summarization.jsonl
230402191648B6MMK2ET_gpt-3.5-turbo_summarization.jsonl
230402191728CPNFOLKJ_gpt-3.5-turbo_summarization.jsonl
230402191843OIHABJ3H_gpt-3.5-turbo_summarization.jsonl
230402191944RDXF3KRI_gpt-3.5-turbo_summarization.jsonl
230402192040I2YBWCIO_gpt-3.5-turbo_summarization.jsonl
230402192213D7GUVP3Y_gpt-3.5-turbo_summarization.jsonl
230402192322SC4ICKM4_gpt-3.5-turbo_summarization.jsonl
230402192359ALA3CIOF_gpt-3.5-turbo_summarization.jsonl
2304021929195ALKDDGD_gpt-3.5-turbo_summarization.jsonl
230402193454ZIN3FZ7Z_gpt-3.5-turbo_summarization.jsonl
230402193606LYPLZZ4H_gpt-3.5-turbo_summarization.jsonl
2304021943265H72SYYT_gpt-3.5-turbo_summarization.jsonl
230402195916YFJVOKOX_gpt-3.5-turbo_summarization.jsonl
230402195939JRE6UXQV_gpt-3.5-turbo_summarization.jsonl
2304022001

In [44]:
import pandas as pd
events = f"/tmp/evallogs/230402193606LYPLZZ4H_gpt-3.5-turbo_summarization.jsonl"

with open(events, "r") as f:
    events_df = pd.read_json(f, lines=True)

In [45]:
for i, r in pd.json_normalize(events_df[events_df.type == "sampling"].data).iterrows():
    print(f"Prompt: {r.prompt[-1]}")
    print(f"Sampled: {r.sampled}")
    print(f"{i}","--" * 25)

Prompt: {'role': 'system', 'content': 'Summarize the following Statement in Arabic: \ufeffالكريات الدموية الحمراء: هي عبارة عن خلاية مقعرة عديمة النواة تحتوي على صبغة الهيموغلوبين أو HB تدعى بخضاب الدم وهي عبارة عن بروتين أحمر اللون يقدر عددها ب4ألى 5 مليون كرية ملم المكعب من الدم تحتوي على كل كرية على 300جزيئة من HB قطرها يصل إلى 7مكرون ويعتبر قطر صغير جدا .\nالخلية الحمراء أو خلية الدم الحمراء (بالإنجليزية: Red blood cell) أو (بالإنجليزية: Erythrocyte) هي إحدى مكونات خلايا الدم و أكثرها عددا. وتحتوى هذه الخلايا على الهيموجلوبين. والهيموجلوبين هو الذي يسمح للخلايا بحمل الأكسجين وثاني أكسيد الكربون، إلى جانب هذه الوظيفة فإن الهيموجلوبين بمثابة الصبغة التي تكسب الدم لونه الأحمر. عندما يكون الدم في الرئتين فإن خضاب الدم-الهيموجلوبين- في كريات الدم الحمراء يتحد مع الأوكسجين ليشكل خضاب الدم المؤكسج .يحمل الدم الأوكسجين إلى الأنسجة حيث يتفكك خضاب الدم المؤكسج ويعطي الخلايا الأوكسجين ويأخذ منها ثاني أوكسيد الكربون الذي يتحد مع الخضاب ليشكل كربامين خضاب الدم وعندما يعود الدم القاتم-المحمل بثا